# Check Boundaries and Initial Behavior of Regional Ocean Model

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

from datetime import datetime
import os
import warnings

from dask.distributed import Client, LocalCluster
import intake
from mom6_tools.MOM6grid import MOM6grid
from mom6_tools.rMOM6_plots import latlon_proj_plot

import xarray as xr

warnings.filterwarnings("ignore")

Basemap module not found. Some regional plots may not function properly


In [3]:
## Default Parameters to be Injected by Papermill
CESM_output_dir = ""
serial = False  # use dask LocalCluster
case_name = ""
savefigs = False
start_date = ""
end_date = ""
mom6_tools_config = {}
lc_kwargs = {}

Setting up paramters how they will be injected by CUPiD workflow for now.

In [4]:
case_name = "CrocCaribGio"
base_case_name = "CrocCaribGio"
CESM_output_dir = "/glade/derecho/scratch/ajanney/archive"
start_date = "2000-01-01"
end_date = "2000-06-30" # maybe should be 2000-07-01
base_start_date = "2000-01-01"
base_end_date = "2000-06-30"
obs_data_dir = None
lc_kwargs = {"threads_per_worker": 1}
serial = True
savefigs = True
mom6_tools_config = {
    "Fnames": {"native": "mom6.h.native.????-??.nc", "static": "mom6.h.static.nc", "sfc": "mom6.h.sfc.????-??.nc"},
    # "oce_cat": "/glade/derecho/scratch/mlevy/temp/reference-datasets.yml" # not for right now
}
subset_kwargs = {}
# product = "/glade/work/ajanney/CUPiD/examples/additional_metrics/computed_notebooks//ocn/Global_Ocean_Surface.ipynb"

In [5]:
OUTDIR = Path(CESM_output_dir) / Path(case_name) / Path("ocn/hist")
print("Output directory is:", OUTDIR)

Output directory is: /glade/derecho/scratch/ajanney/archive/CrocCaribGio/ocn/hist


In [6]:
# The following parameters must be set accordingly
######################################################

# create an empty class object
class args:
    pass


args.start_date = start_date
args.end_date = end_date
args.casename = case_name
args.native = f"{case_name}.{mom6_tools_config['Fnames']['native']}"
args.static = f"{case_name}.{mom6_tools_config['Fnames']['static']}"
args.sfc = f"{case_name}.{mom6_tools_config['Fnames']['sfc']}"
args.mld_obs = "mld-deboyer-tx2_3v2"
args.savefigs = savefigs

In [ ]:
if not os.path.isdir("ncfiles"):
    print("Creating a directory to place netcdf files (ncfiles)... \n")
    os.system("mkdir ncfiles")

In [8]:
# Spin up cluster (if running in parallel)
client = None
if not serial:
    cluster = LocalCluster(**lc_kwargs)
    client = Client(cluster)

client

In [9]:
# load mom6 grid
grd = MOM6grid(OUTDIR / args.static)
grd_xr = MOM6grid(OUTDIR / args.static, xrformat=True)

MOM6 grid successfully loaded... 

MOM6 grid successfully loaded... 



In [10]:
grd_xr

<xarray.Dataset> Size: 33MB
Dimensions:       (xh: 759, yh: 457, time: 1, xq: 760, yq: 458)
Coordinates:
  * xh            (xh) float64 6kB -98.46 -98.38 -98.29 ... -35.71 -35.62 -35.54
  * yh            (yh) float64 4kB -5.958 -5.875 -5.792 ... 31.79 31.88 31.96
  * time          (time) float64 8B 0.0
  * xq            (xq) float64 6kB -98.5 -98.42 -98.33 ... -35.67 -35.58 -35.5
  * yq            (yq) float64 4kB -6.0 -5.917 -5.834 ... 31.83 31.92 32.0
Data variables: (12/24)
    geolon        (yh, xh) float32 1MB -98.46 -98.38 -98.29 ... -35.62 -35.54
    geolat        (yh, xh) float32 1MB ...
    geolon_c      (yq, xq) float32 1MB ...
    geolat_c      (yq, xq) float32 1MB ...
    geolon_u      (yh, xq) float32 1MB -98.5 -98.42 -98.33 ... -35.58 -35.5
    geolat_u      (yh, xq) float32 1MB ...
    ...            ...
    sin_rot       (yh, xh) float32 1MB ...
    cos_rot       (yh, xh) float32 1MB ...
    mgeolon       (yh, xh) float32 1MB -98.46 -98.38 -98.29 ... -35.62 -35.54
    mgeolon_c     (yh, xq) float32 1MB -98.5 -98.42 -98.33 ... -35.58 -35.5
    mgeolon_u     (yh, xq) float32 1MB -98.5 -98.42 -98.33 ... -35.58 -35.5
    mgeolon_v     (yh, xh) float32 1MB -98.46 -98.38 -98.29 ... -35.62 -35.54
Attributes:
    NumFilesInSet:  1
    title:          MOM6 diagnostic fields table for CESM case: CrocCaribGio
    grid_type:      regular
    grid_tile:      N/A

In [33]:
import matplotlib.pyplot as plt
from cartopy import crs as ccrs, feature as cfeature


central_longitude = grd_xr['geolon'].median(dim=['xh','yh']).values

fig = plt.figure(dpi=150, figsize=(12,14))
ax = fig.add_subplot(1,1,1, projection = ccrs.Robinson(central_longitude=central_longitude))
ax.plot()

[]

In [ ]:
print("Reading native dataset...")
startTime = datetime.now()


def preprocess(ds):
    """Compute montly averages and return the dataset with variables"""
    variables = ["tos","sos","SSH", "SSU", "SSV", "speed", "time_bnds"]
    for v in variables:
        if v not in ds.variables:
            ds[v] = xr.zeros_like(ds.SSH)
    return ds[variables]

## I switched to a path object, but it does weird things with the ????-?? field, maybe not worth it?
ds1 = xr.open_mfdataset(str(OUTDIR / args.native), parallel=False)
ds = preprocess(ds1)

print("Time elasped: ", datetime.now() - startTime)

Reading native dataset...
Time elasped:  0:00:00.883178


In [11]:
print("Selecting data between {} and {}...".format(args.start_date, args.end_date))
ds_sel = ds.sel(time=slice(args.start_date, args.end_date))

Selecting data between 2000-01-01 and 2000-06-30...
